### 드라이브 마운트

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/insideout2/dataset/tree_final1.zip -d tree_final

In [ ]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/insideout2/dataset/tree_final2.zip -d tree_final

In [ ]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/insideout2/dataset/tree_final3.zip -d tree_final

In [ ]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/insideout2/dataset/test_img.zip -d test_img

### 데이터 불러오기 및 이미지 증강

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model
from PIL import Image

In [ ]:
label = pd.read_csv('undersampling_data.csv')
label

In [ ]:
label = label.sample(frac=1).reset_index(drop=True)
label

In [ ]:
full_img_path = []

for path in label['img_path']:
    re_path = str(path) + '.png'
    full_img_path.append(re_path)


In [ ]:
label['full_img_path'] = pd.DataFrame(full_img_path)

label

In [ ]:
columns = ['root', 'branch', 'fruit', 'leaves', 'knot']

datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode= 'constant')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(dataframe=label[:835],
                                             directory='tree_final',
                                             x_col='full_img_path',
                                             y_col=columns,
                                             batch_size=32,
                                             seed=2045,
                                             shuffle=True,
                                             class_mode='raw',
                                             target_size=(256, 256))


valid_generator = test_datagen.flow_from_dataframe(dataframe=label[835:1085],
                                             directory='tree_final',
                                             x_col='full_img_path',
                                             y_col=columns,
                                             batch_size=16,
                                             seed=2045,
                                             class_mode='raw',
                                             target_size=(256, 256))

test_generator = test_datagen.flow_from_dataframe(dataframe=label[1085:],
                                             directory='tree_final',
                                             x_col='full_img_path',
                                             y_col=columns,
                                             batch_size=100,
                                             seed=2045,
                                             class_mode='raw',
                                             target_size=(256, 256))

### 모델 설계 및 학습

In [ ]:
initializer = keras.initializers.GlorotNormal()

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(256, 256,3), kernel_initializer=initializer))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=150)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
hist = model.fit(train_generator, steps_per_epoch=26, epochs=300, 
                 validation_data=valid_generator, validation_steps=15,
                 callbacks=[es, mc])

## 학습 결과 분석

### 그래프

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(hist.history['loss']) + 1)

plt.figure(figsize = (12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs, hist.history['loss'])
plt.plot(epochs, hist.history['val_loss'])
plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(epochs, hist.history['acc'])
plt.plot(epochs, hist.history['val_acc'])
plt.title('Training & Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.grid()

plt.show()

### loss, accuracy

In [ ]:
loaded_model = load_model('best_model.h5')

loss, accuracy = loaded_model.evaluate(test_generator, steps = 1)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

### confusion matrix

In [ ]:
classes = np.array(label.columns[0:5])


y_pred = loaded_model.predict(test_generator)
y_pred = (y_pred > 0.5).astype(int)

y_true = test_generator.labels.astype(int)


print(classes)
multilabel_confusion_matrix(y_true, y_pred)

### classification report

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_true, y_pred))